In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from rich.console import Console

console = Console()

# 예제 모델 경로를 정의합니다.
proposer_model_paths = [
    '/path/to/first-model',
    '/path/to/second-model',
    '/path/to/third-model'
]

aggregator_model_path = '/path/to/aggregator-model'  # 집계 모델 경로

def get_least_used_gpu():
    """가장 적게 사용 중인 GPU를 반환하는 함수"""
    device_count = torch.cuda.device_count()
    if device_count == 0:
        raise RuntimeError("No CUDA devices found.")
    
    gpu_memories = [torch.cuda.memory_allocated(i) for i in range(device_count)]
    least_used_gpu = gpu_memories.index(min(gpu_memories))
    
    return torch.device(f"cuda:{least_used_gpu}")

def load_model_and_tokenizer(model_path):
    try:
        model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        
        # 모델이 DataParallel을 사용하고 있는지 확인하고, 그렇다면 model.module으로 접근
        if torch.cuda.device_count() > 1:
            model = torch.nn.DataParallel(model)
        
        return model, tokenizer
    except Exception as e:
        console.log(f"모델 로딩 중 오류 발생: {e}", style="bold red")
        raise

def unload_model_and_tokenizer(model, tokenizer):
    del model
    del tokenizer
    torch.cuda.empty_cache()

def generate_proposals(prompt, model_paths):
    proposals = []
    
    for path in model_paths:
        console.log(f"Loading model from {path}")
        model, tokenizer = load_model_and_tokenizer(path)
        device = get_least_used_gpu()
        model.to(device)
        
        # 모델별 max_length 설정
        if "first-model" in path:
            max_length = 300
        elif "second-model" in path:
            max_length = 2048
        else:
            max_length = 4096  # 다른 모델의 경우
        
        console.log(f"Using max_length: {max_length}")
        try:
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length).to(device)
            console.log(f"Inputs: {inputs['input_ids'].shape}")  # 디버깅을 위한 입력 크기 확인
            with torch.no_grad():
                outputs = model.module.generate(  # DataParallel 사용 시 model.module을 통해 접근
                    **inputs,
                    max_new_tokens=100,
                    num_return_sequences=1,
                    num_beams=3,
                    do_sample=True,
                    top_k=50,
                    temperature=0.7
                )
            responses = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
            console.log(f"Outputs: {[o.shape for o in outputs]}")  # 디버깅을 위한 출력 크기 확인
            proposals.extend(responses)
        except Exception as e:
            console.log(f"모델 생성 중 오류 발생: {e}", style="bold red")
        finally:
            unload_model_and_tokenizer(model, tokenizer)
    
    cleaned_proposals = list(set(proposals))  # 중복 제거
    return cleaned_proposals

def aggregate_responses(proposals, model_path, original_prompt):
    try:
        aggregator_device = get_least_used_gpu()
        model, tokenizer = load_model_and_tokenizer(model_path)
        model.to(aggregator_device)
        
        aggregator_prompt = (
            f"다양한 AI 모델들이 '{original_prompt}' 질문에 대해 생성한 응답들을 종합하여 일관되고 포괄적인 답변을 제공하세요:\n"
            f"{' '.join(proposals)[:1000]}"  # 입력 길이 제한을 더 줄임
        )
        inputs = tokenizer(aggregator_prompt, return_tensors="pt").to(aggregator_device)
        with torch.no_grad():
            outputs = model.module.generate(  # DataParallel 사용 시 model.module을 통해 접근
                **inputs,
                max_new_tokens=300
            )
        final_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    except Exception as e:
        console.log(f"집계 모델 생성 중 오류 발생: {e}", style="bold red")
        final_response = ""
    finally:
        try:
            unload_model_and_tokenizer(model, tokenizer)
        except UnboundLocalError:
            console.log("모델 또는 토크나이저를 언로드하는 중 오류 발생", style="bold red")
    
    return final_response

def main():
    while True:
        question = input("질문을 입력하세요 (종료하려면 'exit' 입력): ")
        if question.lower() == 'exit':
            break

        console.log("Generating proposals for the question")
        proposals = generate_proposals(question, proposer_model_paths)
        console.log(f"Proposals: {proposals}")

        console.log("Aggregating responses")
        final_response = aggregate_responses(proposals, aggregator_model_path, question)
        console.print("제안자 모델의 응답들:", proposals)
        console.print("최종 집계된 응답:", final_response, style="bold green")

if __name__ == "__main__":
    main()


### 코드 설명

이 코드는 여러 개의 제안자 모델(Proposer Models)과 하나의 집계 모델(Aggregator Model)을 사용하여 사용자로부터 받은 질문에 대한 응답을 생성하고, 이를 종합하여 최종 답변을 생성하는 프로그램입니다.

#### 주요 기능
1. **get_least_used_gpu():**
   - 현재 사용 가능한 GPU 중 가장 적게 사용되고 있는 GPU를 선택하여 반환합니다. 이를 통해 모델을 로드하고 계산을 수행할 GPU를 동적으로 선택합니다.

2. **load_model_and_tokenizer(model_path):**
   - 주어진 경로에서 모델과 토크나이저를 로드합니다. 여러 GPU를 사용하는 경우 `DataParallel`을 적용하여 모델을 병렬 처리합니다.

3. **unload_model_and_tokenizer(model, tokenizer):**
   - 사용한 모델과 토크나이저를 메모리에서 해제하고 GPU 캐시를 비웁니다.

4. **generate_proposals(prompt, model_paths):**
   - 여러 제안자 모델을 사용하여 입력된 프롬프트에 대한 응답을 생성합니다. 각 모델은 적절한 GPU에 할당되며, 모델별로 설정된 최대 길이(`max_length`)를 기준으로 입력을 처리합니다.
   - 응답은 중복을 제거하여 반환됩니다.

5. **aggregate_responses(proposals, model_path, original_prompt):**
   - 집계 모델을 사용하여 제안자 모델들로부터 생성된 응답을 종합하고 최종 답변을 생성합니다.
   - 입력된 응답들을 하나의 프롬프트로 결합하여 집계 모델에 입력하고, 최종 결과를 반환합니다.

6. **main():**
   - 사용자로부터 질문을 입력받고, 제안자 모델을 통해 응답을 생성한 후, 집계 모델을 사용하여 최종 답변을 생성합니다.
   - 프로그램은 사용자가 "exit"을 입력할 때까지 반복됩니다.

이 프로그램은 복잡한 모델 간의 협업을 통해 더 일관되고 포괄적인 답변을 생성하는 데 중점을 두고 있으며, 다중 GPU 환경에서의 효율적인 리소스 사용을 고려한 설계가 특징입니다.
